<img src="https://www.python.org/static/community_logos/python-logo-master-v3-TM-flattened.png"/>

# Nützliche Ressourcen

* Die [Python Dokumentation](https://docs.python.org/3/) ist sehr gut; Dokumentation wird in der Community als sehr wichtig angesehen, Sphinx wurde ursprünglich entwickelt, um die Python Doku besser schreiben zu können. Für den Workshop besonders interessant sind:

  * Die [Language Reference](https://docs.python.org/3/reference/index.html) (Syntax, Datenmodell)
  * Die [Library Reference](https://docs.python.org/3/library/index.html) ("Python comes with batteries included". Viele Grundlegende Sachen sind abgedeckt
  
  


# Ablauf

Die ersten Übungen könnt und sollt ihr direkt in diesem Notebook machen; später werden wir noch eine "richtige" Anwendung anpassen, hierzu wird dann zu einer IDE oder Editoren gegriffen



# Hinweise

* IPython/Jupyter Notebooks speichern zwar das Ergebnis eines Befehls ab, aber nicht den Zustand des Systems. Führt daher jede Zelle aus, indem ihr sie anwählt und Strg+Enter drückt. Ansonsten kann es zu fehlern kommen. Wenn ihr z. B. eine Zelle nicht ausgeführt habt, in denen eine Klasse definiert wird, werden darauffolgende Zellen nicht funktionieren, da ja die Klasse fehlt. In dem Fall einfach die Zelle mit der Klassendefinition ausführen und es sollte gehen.

* IPython bietet Typeexpansion (mit Tab) an. Außerdem könnt ihr Shift+Tab drücken, um eine Hilfe zur Funktion angezeigt zu bekommen (sofern IPython weiss, welche Funktion es ist). Zweimal Shift+Tab gibt mehr Hilfe, Dreimal Shift-Tab öffnet einen Frame mit der Doku. ALternativ könnt ihr auch ein oder zwei Fragezeichen hinter den Funktionsnamen schreiben und die Zelle ausführen

* Mit "%psource object" bekommt ihr den Quellcode angezeigt.

## Beispiel

In [1]:
import requests

In [2]:
response = requests.get("https://inoio.de")

In [3]:
%psource response

Falls ihr Inline in Ipython debuggen wollt, macht folgendes:

In [1]:
from IPython.core.debugger import Tracer; debug = Tracer() 

jetzt könnt ihr debug() irgendwo reinschreiben (Klammern nicht vergessen) um den DEbugger zu starten.

Im Debugger könnt ihr normale Pythonexpressions eingeben, und sie weden ausgeführt. mit "cont" gehts dann weiter:

In [8]:
def foo():
    a = 2
    debug()
    return a

In [9]:
foo()

> <ipython-input-8-0ba236d43d68>(4)foo()
      2     a = 2
      3     debug()
----> 4     return a

ipdb> print(a)
2
ipdb> a
ipdb> cont


2

Oder allgemein, wenn eine Exception passiert ist, post-mortem mit `%debug`

In [4]:
def buggy(a):
    inv = 1 / a
    return inv

In [10]:
buggy(0)

ZeroDivisionError: division by zero

In [11]:
%debug

> <ipython-input-4-e9f31ba2c116>(2)buggy()
      1 def buggy(a):
----> 2     inv = 1 / a
      3     return inv

ipdb> inv
*** NameError: name 'inv' is not defined
ipdb> cont


# Funktionen

## Funktionsdefinitionen sind Variablenzuweisungen

Funktionsdefinitionen in Python sind Statements mit Seiteneffekten: sie erzeugen eine neue Funktion und weisen einer Variablen diesen Wert zu. Die folgenden Statements sind mehr oder weniger äquivalent:


In [68]:
def add(x, y):
    return x + y

add_lambda = lambda x, y: x + y

In [69]:
add(1, 2)

3

In [70]:
add_lambda(1, 2)

3

Python ist dynamisch, daher können wir uns einfach mal den Code anschauen, und schauen, ob es unterschiede gibt.

In [91]:
import dis
dis.dis(add)

  2           0 LOAD_FAST                0 (x)
              3 LOAD_FAST                1 (y)
              6 BINARY_ADD
              7 RETURN_VALUE


In [92]:
dis.dis(add_lambda)

  4           0 LOAD_FAST                0 (x)
              3 LOAD_FAST                1 (y)
              6 BINARY_ADD
              7 RETURN_VALUE


In statischen Sprachen bedeutet `add(1, 2)`: "rufe die Funktion `add` mit den Parametern 1 und 2 auf. In Python heißt es: "Hole das Objekt, auf das das Attribute/die Variable `add` zeigt, und rufe dieses mit den Parametern 1 und 2 auf. Wir könnten also auch schreiben:

In [71]:
x = add
x(1, 2)

3

## Übung

Überlege: Was wird passieren, wenn die folgende Zelle ausgeführt wird? Dann führe die Zelle aus. Was ist passiert? War deine Vermutung korrekt? 

In [ ]:
def workshop1(something):
    return "Something was {}".format(something)

# same name as above!
def workshop1(something, something_else):
    return "Something was {} and something else was {}".format(something, something_else)

workshop1("One parameter")

## Keyword arguments, Default Parameter

Wie in Scala auch kann man in Python bei einem Funktionsaufruf explizit den Namen der Parameter mit angeben, das wird bei Funktionen mit vielen Parametern auch empfohlen, da es Dokumentationscharakter hat:

In [103]:
import json
print(json.dumps({"foo": "bar", "bar": "foo"}, sort_keys=True))

{"bar": "foo", "foo": "bar"}


kann man auch verstehen ohne die Doku zu lesen; im Gegensatz zu:

In [104]:
print(json.dumps({"foo": "bar", "bar": "foo"}, False, False, True, True, None, None, None, None, True))

{"bar": "foo", "foo": "bar"}


Varargs gehen auch, Scalas
```scala
def func(data: Any*) = data
```
sieht in Python so aus:

In [62]:
def func(*data):
    return data

Python kennt wie Scala auch (genauer gesagt ist es anderesrum ;)) defaultparameter.

In [74]:
def func_with_default(param = "wert"):
    print("param is", param)
    
func_with_default()

param is wert


### Vorsicht

Python nutzt Early Binding in Defaultparametern, daher keine mutable Werte als Default nehmen, wenn die Parameter verändert werden:

In [75]:
def falsch(lst=[]):
    lst.append(42)
    return lst

In [76]:
falsch()

[42]

In [77]:
falsch()

[42, 42]

### variable Keyword arguments

Anders als in Scala kann in Python eine Funktion aber auch eine beliebe Anzahl von Keyword parametern benutzen, die dann als Map zur Verfügung stehen:

In [63]:
def func(**kwargs):
    return kwargs

In [64]:
func(foo=42, bar="rapunzel")

{'bar': 'rapunzel', 'foo': 42}

Die können natürlich kombiniert werden:

In [67]:
def print_params(a, b, *args, **kwargs):
    print("a is", a)
    print("b is", b)
    print("args is", args)
    print("kwargs is", kwargs)
    
print_params(1, 2, 3, 4, foo=5)

a is 1
b is 2
args is (3, 4)
kwargs is {'foo': 5}


### apply

Manchmal ist es nützlich, die Funktion, die aufgerufen werden soll, und die Paramter dafür getrennt einer anderen Funktion zu übergeben (z. B. Callbacks), um diese Funktion aufzurufen, gibt es dann speziellen Syntax:

In [72]:
args = (1, 2)
add(*args)

3

In [73]:
params = {"b": 42, "foo": 99}
print_params(3, **params)

a is 3
b is 42
args is ()
kwargs is {'foo': 99}


## Übung

Vor Python 2.3 musste man die intere Methode `apply` nutzen, um das gerade gesehene umzusetzen. Seit 2.3 ist sie deprecated, in 3 nicht mehr vorhanden.

Reimplementiere die `apply(object, args, kwargs)` funktion, sie bekommt eine Liste von Argumenten und ein Dictionary mit Keywordargumenten übergeben und ruft dann die entsprechende Methode auf.

Beispiel:
```python
apply(add, (1, 2), {})  # = add(1, 2) = 3
apply(print_params, (1, 2, 3, 4), {"foo": "bar"}) # = print_params(1, 2, 3, 4, foo="bar") 
```

In [78]:
def apply(object, args=(), kwargs={}):
    ...

In [79]:
apply(add, (1, 2))